In [ ]:
# flake8: noqa
# -*- coding: utf-8 -*-
# ---
# jupyter:
#   jupytext:
#     cell_metadata_filter: -all
#     formats: ipynb,py:percent
#     text_representation:
#       extension: .py
#       format_name: percent
#       format_version: '1.3'
#       jupytext_version: 1.17.1
# ---

In [ ]:
"""pinn_2d.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1x0xgwictKM_UcJuvnWKwk2Oj0IWCkJMx

2D PINN model to solve the Poisson problem: -(u_xx+u_yy)=f, with f=sin(pi*x)*sin(pi*y) and homogeneous boundary conditions.
The exact solution is u(x,y)=1/(2*pi^2)*sin(pi*x)*sin(pi*y)
"""

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
from functorch import make_functional, grad, vmap # not really needed for now
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

In [ ]:
"""First we define the Neural Network"""

In [ ]:
from torch import nn

In [ ]:
class PINN(nn.Module):
    def __init__(
        self,
        num_inputs: int = 2,
        num_outputs: int = 1,
        num_hidden: int = 10,
        dim_hidden: int = 15,
        act: nn.Module = nn.Tanh(),
    ) -> None:
        """Simple neural network with linear layers and non-linear activation function
        This class is used as universal function approximator for the solution of
        partial differential equations using PINNs
        Args:
            num_inputs (int, optional): The number of input dimensions
            num_outputs (int, optional): The number of outputs of the model, in general is 1
            num_hidden (int, optional): The number of hidden layers in the model
            dim_hidden (int, optional): The number of neurons for each hidden layer
            act (nn.Module, optional): The type of non-linear activation function to be used
        """
        super().__init__()

        self.layer_in = nn.Linear(num_inputs, dim_hidden)
        self.layer_out = nn.Linear(dim_hidden, num_outputs)

        num_middle = num_hidden - 1
        self.middle_layers = nn.ModuleList(
            [nn.Linear(dim_hidden, dim_hidden) for _ in range(num_middle)]
        )
        self.act = act   # activation function

        self.num_inputs = num_inputs
        self.num_outputs = num_outputs

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.act(self.layer_in(x))
        for layer in self.middle_layers:
            out = self.act(layer(out))
        return self.layer_out(out)

In [ ]:
"""We create the mesh structure that separate the boundaries from the interior of the domain, and it transforms the data into a torch tensor."""

In [ ]:
class Mesh:
      def __init__(self,nx,ny,ax,bx,ay,by):
          self.nx=nx
          self.ny=ny
          self.ax=ax
          self.ay=ay
          self.bx=bx
          self.by=by
          self.int_nodes=nx*ny-(2*nx+2*ny-4)

          x_train = np.linspace(self.ax, self.bx, self.nx)
          y_train = np.linspace(self.ay, self.by, self.ny)
          x_train, y_train = np.meshgrid(x_train, y_train)

          # they cannot have different size bc otherwise vstack does not work
          # boundaries are N-E-S-W (clockwise)
          self.x_train_bc=np.vstack((x_train[0,0:nx-1],x_train[0:ny-1,nx-1].T,x_train[ny-1,1:nx],x_train[1:ny,0].T)).flatten()[:,None]
          self.y_train_bc=np.vstack((y_train[0,0:nx-1],y_train[0:ny-1,nx-1].T,y_train[ny-1,1:nx],y_train[1:ny,0].T)).flatten()[:,None]

          self.x_train_tensor_bc = torch.from_numpy(self.x_train_bc).float()
          self.y_train_tensor_bc = torch.from_numpy(self.y_train_bc).float()

          # internal nodes
          self.x_train_in = x_train[1:ny-1,1:nx-1].flatten()[:,None]
          self.y_train_in = y_train[1:ny-1,1:nx-1].flatten()[:,None]

          self.x_train_tensor_in = torch.from_numpy(self.x_train_in).float()
          self.y_train_tensor_in = torch.from_numpy(self.y_train_in).float()

In [ ]:
"""Definition of the Loss and Train structures."""

In [ ]:
# Source term
def f(x,y):
    return torch.sin(np.pi*x)*torch.sin(np.pi*y)

In [ ]:
# Define the loss function for the PINN
def PINN_loss(model, mesh):
    # interior data
    x=mesh.x_train_tensor_in
    y=mesh.y_train_tensor_in
    x.requires_grad_(True)
    y.requires_grad_(True)
    xy = torch.cat([x, y], dim=1)
    u = model(xy)

    du_dx, = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)
    d2u_dx2, = torch.autograd.grad(du_dx, x, grad_outputs=torch.ones_like(du_dx), create_graph=True)

    du_dy, = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)
    d2u_dy2, = torch.autograd.grad(du_dy, y, grad_outputs=torch.ones_like(du_dy), create_graph=True)

    # interior loss
    interior = d2u_dx2 + d2u_dy2 + f(mesh.x_train_tensor_in,mesh.y_train_tensor_in)

    # boundary data
    x_boundary=mesh.x_train_tensor_bc
    y_boundary=mesh.y_train_tensor_bc
    x_boundary.requires_grad_(True)
    y_boundary.requires_grad_(True)
    xy_boundary = torch.cat([x_boundary, y_boundary], dim=1)
    u_boundary = model(xy_boundary)

    # boundary loss
    boundary = u_boundary

    loss = nn.MSELoss()
    loss_value=loss(interior, torch.zeros_like(interior)) + loss(boundary, torch.zeros_like(boundary))

    return loss_value

In [ ]:
# Define the training loop
def train(model, mesh, iterations, learning_rate):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for i in range(iterations):
        optimizer.zero_grad() # we need to set to zero the gradients of all model parameters (PyTorch accumulates grad by default)
        loss = PINN_loss(model, mesh) # compute the loss value for the current batch of data
        loss.backward() # backpropagation to compute gradients of model param respect to the loss. computes dloss/dx for every parameter x which has requires_grad=True.
        optimizer.step() # update the model param doing an optim step using the computed gradients and learning rate.

        if (i+1) % 100 == 0:
            print(f"Iteration {i+1}/{iterations}, Loss: {loss.item()}")

In [ ]:
"""# Training and Test section"""

In [ ]:
# Generate training data
# nx and ny need to be the same at the moment (problems generated with vstack in the mesh structure)
nx=20 # number of point in x axis
ny=20 # number of points in y axis

In [ ]:
# Domain is Omega=[ax,bx]x[ay,by]
# Interval [a,b] along the x axis
ax=0
bx=1
# Interval [a,b] along the y axis
ay=0
by=1

In [ ]:
mesh=Mesh(nx,ny,ax,bx,ay,by)

In [ ]:
# Create an instance of the PINN model
model = PINN()

In [ ]:
# Train the PINN model
model.train() # this tells pytorch to update the weigths
train(model, mesh, iterations=1000, learning_rate=0.001)

In [ ]:
# Test the PINN (the test includes the boundary!)
x_test = np.linspace(0, 1, 10)
y_test = np.linspace(0, 1, 10)
x_test, y_test = np.meshgrid(x_test, y_test)
x_test_tensor = torch.from_numpy(x_test.flatten()[:,None]).float()
y_test_tensor = torch.from_numpy(y_test.flatten()[:,None]).float()
xy_test = torch.cat([x_test_tensor, y_test_tensor], dim=1)
model.eval()
prediction = model.forward(xy_test)

In [ ]:
# Transoform the prediction into a numpy object for the plot
prediction_np= prediction.detach().numpy()

In [ ]:
"""Plot of the predicted solution"""

In [ ]:
def plot_solution(X_star, u_star):

    a = X_star.min(0)
    b = X_star.max(0)

    nn = 200
    x = np.linspace(0, 1, nn)
    y = np.linspace(0, 1, nn)
    X, Y = np.meshgrid(x,y)

    # interpolate the solution
    U_star = griddata(X_star, u_star.flatten(), (X, Y), method='cubic')

    plt.figure()
    plt.pcolor(X,Y,U_star, cmap = 'jet')
    plt.colorbar()

In [ ]:
plot_solution(xy_test,prediction_np)

In [ ]:
"""Plot of the exact solution"""

In [ ]:
# Exact solution
def u_exact(x,y):
    return 1/(2*np.pi**2)*torch.sin(np.pi*x)*torch.sin(np.pi*y)
plot_solution(xy_test,u_exact(x_test_tensor,y_test_tensor))